In [1]:
import os

import torch
import torch.nn as nn

from torch import optim
from torch.utils.data import random_split
from nscl.datasets.clevr_dataset import build_clevr_dataset, build_clevr_dataloader, CLEVRCurriculumSampler
from nscl.datasets.clevr_definition import CLEVRDefinition
from nscl.models.nscl_module import NSCLModule
from tqdm.autonotebook import tqdm

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
train_img_root = os.path.abspath(os.getcwd()) + '/data/images'
train_scene_json = os.path.abspath(os.getcwd())  + '/data/scenes.json'
train_question_json = os.path.abspath(os.getcwd())  + '/data/questions.json'

dataset = build_clevr_dataset(train_img_root, train_scene_json, train_question_json)
train_ratio, test_ratio = 0.8, 0.2
train_dataset, test_dataset = random_split(dataset, [int(train_ratio * len(dataset)), int(test_ratio * len(dataset))])

loading scenes from: /home/tpanyapiang/git/MSc/nscl_reproducability_challenge/data/scenes.json
loading questions from: /home/tpanyapiang/git/MSc/nscl_reproducability_challenge/data/questions.json


In [6]:
batch_size = 128
num_workers = 1
num_epoch = 10

curriculum_sampler = CLEVRCurriculumSampler(train_dataset, max_scene_size=5, max_program_size=7, max_data_size=1000)
trainloader = build_clevr_dataloader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False,drop_last=False, sampler=curriculum_sampler)

model = NSCLModule(CLEVRDefinition.attribute_concept_map)
mse_loss = nn.MSELoss()
bce_loss = nn.BCELoss()
ce_loss = nn.CrossEntropyLoss()
# Freeze visual module
for param in model.visual_module.parameters():
    param.requires_grad = False

optimiser = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.01)

for i in tqdm(range(num_epoch)):
    for images, questions, scenes in trainloader:
        results = model(images, questions, scenes)
        # Compute loss
        for i, q in enumerate(questions):
            predicted_answer = results[i]
            true_answer = q.answer_tensor
            if isinstance(q.answer, bool):
                loss_function = bce_loss
                predicted_answer = torch.stack([predicted_answer, torch.tensor(1 - predicted_answer.item())])
            elif isinstance(q.answer, int):
                loss_function = mse_loss
            elif q.answer_tensor.shape[0] == 2:
                loss_function = bce_loss
            else:
                loss_function = ce_loss
                predicted_answer = predicted_answer.unsqueeze(0)

            # print(f'{loss_function}|{q.raw_question},{q.answer},{true_answer.data},{predicted_answer.data}')
            try:
                loss = loss_function(predicted_answer, true_answer)
                loss.backward()
            except Exception as ex:
                print(f'Unsupport question : {q.raw_question}')
                print(f'Not compute loss')
    optimiser.step()


Preparing curriculum sampler....


Operator not implemented count_less
Operator not implemented count_less
Unsupport question : Is the number of gray objects less than the number of metal objects?
Not compute loss
Operator not implemented count_less
Unsupport question : Is the number of gray objects less than the number of metal objects?
Not compute loss
Operator not implemented count_less
Unsupport question : Is the number of gray objects less than the number of metal objects?
Not compute loss
Operator not implemented count_less
Unsupport question : Is the number of gray objects less than the number of metal objects?
Not compute loss
Operator not implemented count_less
Unsupport question : Is the number of gray objects less than the number of metal objects?
Not compute loss
Operator not implemented count_less
Unsupport question : Is the number of gray objects less than the number of metal objects?
Not compute loss
Operator not implemented count_less
Unsupport question : Is the number of gray objects less than the numbe